# "Smooth" statistics & smooth line plot

 Statistics "smooth" are calculated on the sample of two continuous variables (i.e., sample of points or line); It interpolates data points to create a smoother curve.

This notebook uses definitions from [DataFrame](https://kotlin.github.io/dataframe/overview.html).

## Usage

"Smooth" statistic is useful in case of the presence of overplotting or noises, making it easier to discern underlying trends and patterns. It can also be used to make a more pretty line with a small number of points.

## Arguments

* Input (mandatory):
  - `x` - numeric sample of input points `x` coordinates;
  - `y` - numeric sample of input points `y` coordinates;
* Parameters (optional):
  - `method: SmoothMethod` - smoothing model:
    - `SmoothMethod.Linear(confidenceLevel: Double)` -  linear model;
    - `SmoothMethod.Polynomial(degree: Int, confidenceLevel: Double)` -  polynomial model;
    - `SmoothMethod.LOESS(span: Double, loessCriticalSize: Int, samplingSeed: Long, confidenceLevel: Double)` -  Local Polynomial Regression model;
  - `smootherPointCount: Int` - number of sampled points;

### Generalized signature

The specific signature depends on the function, but all functions related to "smooth" statistic (which will be discussed further below - different variations of `statSmooth()`, `smoothLine()`) have approximately the same signature with the arguments above:

```
statSmoothArgs := 
   x, 
   y,
   method: SmoothMethod = SmoothMethod.LOESS(),
   smootherPointCount: Int = 100
```

The possible types of `x` and `y` depend on where a certain function is used. They can be simply `Iterable` (`List`, `Set`, etc.) or a reference to a column in a `DataFrame` (`String`, `ColumnAccessor`) or the `DataColumn` itself.

## Output statistics

| name      | type   | description                                         |
|-----------|--------|-----------------------------------------------------|
| Stat.x    | Double | `x` coordinate                                      |
| Stat.y    | Double | `y` coordinate                                      |
| Stat.yMin | Double | Lower pointwise confidence interval around the mean |
| Stat.yMax | Double | Upper pointwise confidence interval around the mean |
| Stat.se   | Double | Standard error                                      |

## StatSmooth plots

In [1]:
%use dataframe(0.12.0)
%use kandy(0.4.5-dev-30)
@file:Repository("https://packages.jetbrains.team/maven/p/kds/kotlin-ds-maven")
@file:DependsOn("org.jetbrains.kotlinx:kotlin-statistics-jvm:0.0.0-dev-11")

In [2]:
// to generate the data we use a standard java math library
// https://commons.apache.org/proper/commons-math/
import org.apache.commons.math3.distribution.NormalDistribution
import org.apache.commons.math3.distribution.UniformRealDistribution
import kotlin.random.Random

// generate line with formula
val xs = (-100..100).map { it / 50.0 }
val lineFormula = {x: Double -> 2.0 / (x * x + 0.5)}
// generate noises from normal distribution
val noises = NormalDistribution(0.0, 0.1).sample(xs.size).toList()
val ys = xs.zip(noises).map { lineFormula(it.first) + it.second }
// and drop 2/3 points
val (newXs, newYs) = xs.zip(ys).shuffled(Random(17)).take(xs.size * 1 / 3).sortedBy { it.first }.unzip()

In [3]:
// gather them into the DataFrame
val df = dataFrameOf(
    "speed" to newXs,
    "efficiency" to newYs
)

In [4]:
df.head(5)

<iframe onload="o_resize_iframe_out_1()" style="width:100%;" class="result_container" id="iframe_out_1" frameBorder="0" srcdoc=" <html>
 <head>
 <style type="text/css">
 :root {
 --background: #fff;
 --background-odd: #f5f5f5;
 --background-hover: #d9edfd;
 --header-text-color: #474747;
 --text-color: #848484;
 --text-color-dark: #000;
 --text-color-medium: #737373;
 --text-color-pale: #b3b3b3;
 --inner-border-color: #aaa;
 --bold-border-color: #000;
 --link-color: #296eaa;
 --link-color-pale: #296eaa;
 --link-hover: #1a466c;
}

:root[theme="dark"], :root [data-jp-theme-light="false"], .dataframe_dark{
 --background: #303030;
 --background-odd: #3c3c3c;
 --background-hover: #464646;
 --header-text-color: #dddddd;
 --text-color: #b3b3b3;
 --text-color-dark: #dddddd;
 --text-color-medium: #b2b2b2;
 --text-color-pale: #737373;
 --inner-border-color: #707070;
 --bold-border-color: #777777;
 --link-color: #008dc0;
 --link-color-pale: #97e1fb;
 --link-hover: #00688e;
}

p.dataframe_description {
 color: var(--text-color-dark);
}

table.dataframe {
 font-family: "Helvetica Neue", Helvetica, Arial, sans-serif;
 font-size: 12px;
 background-color: var(--background);
 color: var(--text-color-dark);
 border: none;
 border-collapse: collapse;
}

table.dataframe th, td {
 padding: 6px;
 border: 1px solid transparent;
 text-align: left;
}

table.dataframe th {
 background-color: var(--background);
 color: var(--header-text-color);
}

table.dataframe td {
 vertical-align: top;
}

table.dataframe th.bottomBorder {
 border-bottom-color: var(--bold-border-color);
}

table.dataframe tbody > tr:nth-child(odd) {
 background: var(--background-odd);
}

table.dataframe tbody > tr:nth-child(even) {
 background: var(--background);
}

table.dataframe tbody > tr:hover {
 background: var(--background-hover);
}

table.dataframe a {
 cursor: pointer;
 color: var(--link-color);
 text-decoration: none;
}

table.dataframe tr:hover > td a {
 color: var(--link-color-pale);
}

table.dataframe a:hover {
 color: var(--link-hover);
 text-decoration: underline;
}

table.dataframe img {
 max-width: fit-content;
}

table.dataframe th.complex {
 background-color: var(--background);
 border: 1px solid var(--background);
}

table.dataframe .leftBorder {
 border-left-color: var(--inner-border-color);
}

table.dataframe .rightBorder {
 border-right-color: var(--inner-border-color);
}

table.dataframe .rightAlign {
 text-align: right;
}

table.dataframe .expanderSvg {
 width: 8px;
 height: 8px;
 margin-right: 3px;
}

table.dataframe .expander {
 display: flex;
 align-items: center;
}

/* formatting */

table.dataframe .null {
 color: var(--text-color-pale);
}

table.dataframe .structural {
 color: var(--text-color-medium);
 font-weight: bold;
}

table.dataframe .dataFrameCaption {
 font-weight: bold;
}

table.dataframe .numbers {
 color: var(--text-color-dark);
}

table.dataframe td:hover .formatted .structural, .null {
 color: var(--text-color-dark);
}

table.dataframe tr:hover .formatted .structural, .null {
 color: var(--text-color-dark);
}




:root {
 --scroll-bg: #f5f5f5;
 --scroll-fg: #b3b3b3;
}
:root[theme="dark"], :root [data-jp-theme-light="false"]{
 --scroll-bg: #3c3c3c;
 --scroll-fg: #97e1fb;
}
body {
 scrollbar-color: var(--scroll-fg) var(--scroll-bg);
}
body::-webkit-scrollbar {
 width: 10px; /* Mostly for vertical scrollbars */
 height: 10px; /* Mostly for horizontal scrollbars */
}
body::-webkit-scrollbar-thumb {
 background-color: var(--scroll-fg);
}
body::-webkit-scrollbar-track {
 background-color: var(--scroll-bg);
}
 </style>
 </head>
 <body>
 
<table class="dataframe" id="df_939524096"></table>

<p class="dataframe_description">DataFrame: rowsCount = 5, columnsCount = 2</p>

 </body>
 <script>
 (function () {
 window.DataFrame = window.DataFrame || new (function () {
 this.addTable = function (df) {
 let cols = df.cols;
 for (let i = 0; i < cols.length; i++) {
 for (let c of cols[i].children) {
 cols[c].parent = i;
 }
 }
 df.nrow = 0
 for (let i = 0; i 

`df` has a signature

<table>
  <thead>
    <tr>
      <th>speed</th>
      <th>efficiency</th>
    </tr>
  </thead>
</table>

Let's take a look at `StatSmooth` output DataFrame:

In [5]:
df.statSmooth("speed", "efficiency").head(5)

<iframe onload="o_resize_iframe_out_2()" style="width:100%;" class="result_container" id="iframe_out_2" frameBorder="0" srcdoc=" <html>
 <head>
 <style type="text/css">
 :root {
 --background: #fff;
 --background-odd: #f5f5f5;
 --background-hover: #d9edfd;
 --header-text-color: #474747;
 --text-color: #848484;
 --text-color-dark: #000;
 --text-color-medium: #737373;
 --text-color-pale: #b3b3b3;
 --inner-border-color: #aaa;
 --bold-border-color: #000;
 --link-color: #296eaa;
 --link-color-pale: #296eaa;
 --link-hover: #1a466c;
}

:root[theme="dark"], :root [data-jp-theme-light="false"], .dataframe_dark{
 --background: #303030;
 --background-odd: #3c3c3c;
 --background-hover: #464646;
 --header-text-color: #dddddd;
 --text-color: #b3b3b3;
 --text-color-dark: #dddddd;
 --text-color-medium: #b2b2b2;
 --text-color-pale: #737373;
 --inner-border-color: #707070;
 --bold-border-color: #777777;
 --link-color: #008dc0;
 --link-color-pale: #97e1fb;
 --link-hover: #00688e;
}

p.dataframe_description {
 color: var(--text-color-dark);
}

table.dataframe {
 font-family: "Helvetica Neue", Helvetica, Arial, sans-serif;
 font-size: 12px;
 background-color: var(--background);
 color: var(--text-color-dark);
 border: none;
 border-collapse: collapse;
}

table.dataframe th, td {
 padding: 6px;
 border: 1px solid transparent;
 text-align: left;
}

table.dataframe th {
 background-color: var(--background);
 color: var(--header-text-color);
}

table.dataframe td {
 vertical-align: top;
}

table.dataframe th.bottomBorder {
 border-bottom-color: var(--bold-border-color);
}

table.dataframe tbody > tr:nth-child(odd) {
 background: var(--background-odd);
}

table.dataframe tbody > tr:nth-child(even) {
 background: var(--background);
}

table.dataframe tbody > tr:hover {
 background: var(--background-hover);
}

table.dataframe a {
 cursor: pointer;
 color: var(--link-color);
 text-decoration: none;
}

table.dataframe tr:hover > td a {
 color: var(--link-color-pale);
}

table.dataframe a:hover {
 color: var(--link-hover);
 text-decoration: underline;
}

table.dataframe img {
 max-width: fit-content;
}

table.dataframe th.complex {
 background-color: var(--background);
 border: 1px solid var(--background);
}

table.dataframe .leftBorder {
 border-left-color: var(--inner-border-color);
}

table.dataframe .rightBorder {
 border-right-color: var(--inner-border-color);
}

table.dataframe .rightAlign {
 text-align: right;
}

table.dataframe .expanderSvg {
 width: 8px;
 height: 8px;
 margin-right: 3px;
}

table.dataframe .expander {
 display: flex;
 align-items: center;
}

/* formatting */

table.dataframe .null {
 color: var(--text-color-pale);
}

table.dataframe .structural {
 color: var(--text-color-medium);
 font-weight: bold;
}

table.dataframe .dataFrameCaption {
 font-weight: bold;
}

table.dataframe .numbers {
 color: var(--text-color-dark);
}

table.dataframe td:hover .formatted .structural, .null {
 color: var(--text-color-dark);
}

table.dataframe tr:hover .formatted .structural, .null {
 color: var(--text-color-dark);
}




:root {
 --scroll-bg: #f5f5f5;
 --scroll-fg: #b3b3b3;
}
:root[theme="dark"], :root [data-jp-theme-light="false"]{
 --scroll-bg: #3c3c3c;
 --scroll-fg: #97e1fb;
}
body {
 scrollbar-color: var(--scroll-fg) var(--scroll-bg);
}
body::-webkit-scrollbar {
 width: 10px; /* Mostly for vertical scrollbars */
 height: 10px; /* Mostly for horizontal scrollbars */
}
body::-webkit-scrollbar-thumb {
 background-color: var(--scroll-fg);
}
body::-webkit-scrollbar-track {
 background-color: var(--scroll-bg);
}
 </style>
 </head>
 <body>
 
<table class="dataframe" id="df_939524097"></table>

<p class="dataframe_description">DataFrame: rowsCount = 5, columnsCount = 1</p>

 </body>
 <script>
 (function () {
 window.DataFrame = window.DataFrame || new (function () {
 this.addTable = function (df) {
 let cols = df.cols;
 for (let i = 0; i < cols.length; i++) {
 for (let c of cols[i].children) {
 cols[c].parent = i;
 }
 }
 df.nrow = 0
 for (let i = 0; i 

It has the following signature:

<table>
  <thead>
    <tr>
      <th alignt="left" colspan="5">Stat</th>
    </tr>
  </thead>
  <thead>
    <tr>
      <th>x</th>
      <th>y</th>
      <th>yMin</th>
      <th>yMax</th>
      <th>se</th>
    </tr>
  </thead>
</table>

As you can see, we got a `DataFrame` with one `ColumnGroup` called `Stat` which contains several columns with statics. For `statSmooth`, each row corresponds to one of line points. `Stat.x` is the column with this point `x` coordinate; `Stat.y` is points `y` coordinate; `Stat.yMin` - lower point of confidence level; `Stat.yMax` - upper point of confidence level; `Stat.se` - standard error.
`DataFrame` with "smooth" statistics is called `StatSmoothFrame`

### `statSmooth` transform

`statSmooth(statSmoothArgs) { /*new plotting context*/ }` modifies a plotting context - instead of original data (no matter was it empty or not) new `StatSmooth` dataset (calculated on given arguments; inputs can be provided as `Iterable` or as dataset column reference - by name as a `String`, as a `ColumnReference` or as a `DataColumn`) is used inside a new context (original dataset and primary context are not affected - you can add layers using initial dataset outside the `statSmooth` context). Since the old dataset is irrelevant, we cannot use references for its columns. But we can refer to the new ones. They are all contained in the `Stat` group and can be called inside the new context:

In [6]:
plot {
    statSmooth(newXs, newYs) {
        // new `StatSmooth` dataset here
        area {
            // use `Stat.*` columns for mappings
            x(Stat.x)
            y(Stat.y)
        }
    }
    points {
        x(newXs)
        y(newYs)
    }
} 

-2.0
 
 
 
 
 
 
 
 
 -1.5
 
 
 
 
 
 
 
 
 -1.0
 
 
 
 
 
 
 
 
 -0.5
 
 
 
 
 
 
 
 
 0.0
 
 
 
 
 
 
 
 
 0.5
 
 
 
 
 
 
 
 
 1.0
 
 
 
 
 
 
 
 
 1.5
 
 
 
 
 
 
 
 
 2.0
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0.0
 
 
 
 
 
 
 0.5
 
 
 
 
 
 
 1.0
 
 
 
 
 
 
 1.5
 
 
 
 
 
 
 2.0
 
 
 
 
 
 
 2.5
 
 
 
 
 
 
 3.0
 
 
 
 
 
 
 3.5
 
 
 
 
 
 
 4.0
 
 
 
 
 
 
 
 
 
 
 
 
 <path d="M24.54387721725996 331.27768702243384 L24.54387721725996 331.27768702243384 L29.50223625104985 326.93064749217064 L34.46059528483974 322.5638312976077 L39.4189543186296 318.1618294688476 L44.37731335241949 313.7256867876034 L49.33567238620938 309.25910234876 L54.29403141999927 304.76151078753924 L59.25239045378916 300.2325207616638 L64.21074948757905 295.6727227105422 L69.16910852136894 291.0826588754476 L74.1274675551588 286.4628555678135 L79.08582658894869 281.814429503061 L84.04418562273858 277.1385681516668 L89.0025446565285 272.4356312336007 L93.96090369031839 267.7070034332179 L98.91926272410825 262.95693608688964 L103.87762175789817 258.190163912966 L108.83598079168803 253.41142162979696 L113.79433982547792 248.62544395573246 L118.75269885926781 243.8373521494379 L123.7110578930577 239.05715674199286 L128.6694169268476 234.30455832871 L133.62777596063745 229.59232827935034 L138.58613499442737 224.908884194346 L143.54449402821723 220.2389933529069 L148.50285306200712 215.53504533910203 L153.461212095797 210.44117091971896 L158.4195711295869 204.04216697901958 L163.3779301633768 197.1561785738488 L168.33628919716665 191.45474393655962 L173.29464823095654 185.208397101499 L178.25300726474643 177.37192345372614 L183.21136629853635 168.95206892745927 L188.1697253323262 160.56873089278992 L193.12808436611613 151.69598934185515 L198.086443399906 142.79515631996503 L203.04480243369585 134.18208988771647 L208.00316146748577 125.23348794991577 L212.96152050127563 115.34380522696387 L217.91987953506555 105.61123859645525 L222.8782385688554 96.75983597882367 L227.83659760264533 91.28696731887607 L232.7949566364352 94.52222260484922 L237.75331567022508 102.1605820234069 L242.71167470401497 108.990197741294 L247.67003373780486 112.77166902239387 L252.62839277159475 114.7422478456038 L257.5867518053846 116.32608544024163 L262.54511083917447 118.66969536009881 L267.5034698729644 122.61934635651454 L272.46182890675425 126.2155489035541 L277.4201879405442 123.08458903535848 L282.3785469743341 122.99391483812028 L287.33690600812395 125.69800074196942 L292.29526504191386 126.74543864270834 L297.2536240757037 131.2412518953204 L302.21198310949364 134.89200573301386 L307.1703421432835 140.27444647211018 L312.12870117707337 146.86721890998825 L317.0870602108633 150.1868336110628 L322.04541924465315 146.35125251678716 L327.00377827844306 141.6426182161631 L331.9621373122329 141.92276721865298 L336.92049634602284 148.94535007709266 L341.8788553798127 157.06686316660964 L346.83721441360257 163.97438693525575 L351.7955734473925 170.9964682051072 L356.75393248118235 178.12030878367622 L361.71229151497226 184.99403637480933 L366.6706505487621 191.64997635406093 L371.62900958255204 198.1269794733293 L376.5873686163419 204.46389648451247 L381.54572765013177 210.69494871921114 L386.5040866839217 216.81068409264856 L391.46244571771155 222.8570895299176 L396.42080475150146 228.69602905000318 L401.3791637852913 233.9845709063063 L406.3375228190812 238.62804266876446 L411.2958818528711 243.53948776998465 L416.25424088666097 248.3911897063689 L421.2125999204509 252.77912680639034 L426.17095895424075 257.13161882868974 L431.12931798803066 261.4346692546688 L436.0876770218205 265.68315777457804 L441.0460360556104 269.8916173025476 L446.0043950894003 274.05977788760725 L450.96275412319017 278.1870425236739 L455.9211131569801 282.27545209235245 L460.87947219077 286.3247449034544 L465.83783122455986 290.3341277011907 L470.7961902583497 294.30268511009103 L475.7545492921396 298.2298109178349 L480.71290832592956 302.1151224188144 L485.6712673597194 305.96021824948787 

In [7]:
df.plot { 
    statSmooth(speed, efficiency, method = SmoothMethod.Polynomial(2), smootherPointCount = 250) {
        ribbon { 
            x(Stat.x)
            yMin(Stat.yMin)
            yMax(Stat.yMax)
         }
    }
    // dataset is noe changed here
    points {
        x(speed)
        y(efficiency)
    }
 }

-2.0
 
 
 
 
 
 
 
 
 -1.5
 
 
 
 
 
 
 
 
 -1.0
 
 
 
 
 
 
 
 
 -0.5
 
 
 
 
 
 
 
 
 0.0
 
 
 
 
 
 
 
 
 0.5
 
 
 
 
 
 
 
 
 1.0
 
 
 
 
 
 
 
 
 1.5
 
 
 
 
 
 
 
 
 2.0
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 -1
 
 
 
 
 
 
 0
 
 
 
 
 
 
 1
 
 
 
 
 
 
 2
 
 
 
 
 
 
 3
 
 
 
 
 
 
 4
 
 
 
 
 
 
 
 
 
 
 
 
 <path d="M24.77449041313926 279.8326793582822 L24.77449041313926 279.8326793582822 L26.764409321825156 276.5244384419523 L28.75432823051102 273.2439248637258 L30.744247139196915 269.99113440306667 L32.73416604788278 266.76606274248485 L34.7240849565687 263.5687054650253 L36.71400386525457 260.39905805169093 L38.70392277394046 257.25711587879755 L40.69384168262633 254.1428742152601 L42.68376059131222 251.05632821980788 L44.67367949999809 247.9974729381283 L46.66359840868398 244.96630329993667 L48.653517317369875 241.96281411597198 L50.64343622605574 238.98700007491618 L52.633355134741635 236.0388557402364 L54.6232740434275 233.11837554694864 L56.61319295211342 230.22555379830175 L58.60311186079926 227.36038466238077 L60.59303076948518 224.52286216862794 L62.58294967817105 221.71298020428176 L64.57286858685694 218.93073251073153 L66.5627874955428 216.17611267978808 L68.5527064042287 213.44911414986922 L70.54262531291457 210.74973020209984 L72.53254422160046 208.07795395632587 L74.52246313028635 205.43377836704263 L76.51238203897222 202.8171962192368 L78.50230094765814 200.22820012414257 L80.49221985634398 197.66678251491325 L82.4821387650299 195.13293564220663 L84.47205767371577 192.62665156968865 L86.46197658240166 190.14792216945307 L88.45189549108753 187.69673911736143 L90.44181439977342 185.2730938883035 L92.43173330845929 182.87697775138173 L94.42165221714518 180.5083817650219 L96.4115711258311 178.16729677201334 L98.40149003451694 175.85371339448267 L100.39140894320286 173.56762202880452 L102.3813278518887 171.3090128404559 L104.37124676057462 169.07787575881713 L106.36116566926049 166.87420047192822 L108.35108457794638 164.6979764212053 L110.34100348663225 162.5491927961264 L112.33092239531814 160.42783852889391 L114.320841304004 158.33390228908422 L116.3107602126899 156.26737247829385 L118.3006791213758 154.22823722479401 L120.29059803006166 152.21648437820608 L122.28051693874758 150.23210150421096 L124.27043584743342 148.27507587930816 L126.26035475611934 146.34539448563916 L128.2502736648052 144.44304400589436 L130.2401925734911 142.56801081832046 L132.23011148217697 140.72028099185013 L134.22003039086286 138.89984028137513 L136.20994929954873 137.10667412318648 L138.19986820823462 135.34076763060685 L140.1897871169205 133.60210558984272 L142.17970602560638 131.89067245608425 L144.1696249342923 130.20645234988382 L146.15954384297814 128.54942905384704 L148.14946275166403 126.9195860096685 L150.13938166034993 125.31690631555199 L152.12930056903582 123.74137272405065 L154.11921947772169 122.19296764037068 L156.10913838640758 120.67167312118002 L158.09905729509347 119.17747087396683 L160.08897620377934 117.7103422569958 L162.0788951124652 116.27026827991031 L164.06881402115113 114.85722960503179 L166.058732929837 113.47120654940915 L168.0486518385229 112.11217908767213 L170.03857074720878 110.78012685574518 L172.02848965589465 109.47502915547915 L174.0184085645805 108.19686496025889 L176.0083274732664 106.9456129216465 L177.9982463819523 105.72125137712027 L179.98816529063816 104.5237583589695 L181.97808419932406 103.3531116044052 L183.96800310800995 102.20928856694638 L185.95792201669585 101.09226642914058 L187.94784092538174 100.00202211667545 L189.9377598340676 98.9385323139364 L191.92767874275347 97.90177348106218 L193.91759765143937 96.89172187254744 L195.90751656012526 95.90835355743621 L197.89743546881112 94.95164444114678 L199.887354377497 94.0215702889607 L201.8772732861829 93.11810675120458 L203.86719219486878 92.24122939014336 L205.8571111035547 91.3909137085978 L207.84703001224057 90.56713518028832 L209.83694892092643 89.76986928189706 L211.82686782961233 88.99909152683117 L213.81678673829822

### `smoothLine` layer

`smoothLine` layer is a shortcut for fast plotting a smoothed line:

In [8]:
val smoothLineLayerPlot = plot {
    smoothLine(newXs, newYs)
    layout.title = "`smoothLine()` layer"
}
smoothLineLayerPlot

-2.0
 
 
 
 
 
 
 
 
 -1.5
 
 
 
 
 
 
 
 
 -1.0
 
 
 
 
 
 
 
 
 -0.5
 
 
 
 
 
 
 
 
 0.0
 
 
 
 
 
 
 
 
 0.5
 
 
 
 
 
 
 
 
 1.0
 
 
 
 
 
 
 
 
 1.5
 
 
 
 
 
 
 
 
 2.0
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0.5
 
 
 
 
 
 
 1.0
 
 
 
 
 
 
 1.5
 
 
 
 
 
 
 2.0
 
 
 
 
 
 
 2.5
 
 
 
 
 
 
 3.0
 
 
 
 
 
 
 
 
 
 
 
 
 <path d="M24.54387721725996 305.4545454545455 L24.54387721725996 305.4545454545455 L29.50223625104985 300.1852028755624 L34.46059528483974 294.8918876553049 L39.4189543186296 289.5559215322541 L44.37731335241949 284.17857095745757 L49.33567238620938 278.7643198556748 L54.29403141999927 273.31248290976583 L59.25239045378916 267.8225857471122 L64.21074948757905 262.29534407952053 L69.16910852136894 256.7314151945559 L74.1274675551588 251.13143707014044 L79.08582658894869 245.49676335306657 L84.04418562273858 239.82883345714708 L89.0025446565285 234.12808342344846 L93.96090369031839 228.39619172215268 L98.91926272410825 222.63831168752392 L103.87762175789817 216.8601825940333 L108.83598079168803 211.06754371615202 L113.79433982547792 205.26613432835114 L118.75269885926781 199.46216225693536 L123.7110578930577 193.66776194818448 L128.6694169268476 187.90681383497454 L133.62777596063745 182.19479896412034 L138.58613499442737 176.51767752052825 L143.54449402821723 170.85698488622222 L148.50285306200712 165.15500923453905 L153.461212095797 158.9803772388593 L158.4195711295869 151.2237088918001 L163.3779301633768 142.87673351944747 L168.33628919716665 135.96563641844233 L173.29464823095654 128.3940141426522 L178.25300726474643 118.89489089479247 L183.21136629853635 108.68861196513777 L188.1697253323262 98.52659716114152 L193.12808436611613 87.7713430344734 L198.086443399906 76.98203732327724 L203.04480243369585 66.54155308771996 L208.00316146748577 55.69434342124407 L212.96152050127563 43.7063856624315 L217.91987953506555 31.908879016379956 L222.8782385688554 21.179491263611055 L227.83659760264533 14.545454545454561 L232.7949566364352 18.467127747084305 L237.75331567022508 27.726103249279788 L242.71167470401497 36.00474544929119 L247.67003373780486 40.58852424724813 L252.62839277159475 42.977197003981416 L257.5867518053846 44.89707438662714 L262.54511083917447 47.73792353309966 L267.5034698729644 52.52556441435854 L272.46182890675425 56.88476636590269 L277.4201879405442 53.08951674102258 L282.3785469743341 52.97960437307745 L287.33690600812395 56.25741100243596 L292.29526504191386 57.52708179580691 L297.2536240757037 62.97676313277668 L302.21198310949364 67.40209041779892 L307.1703421432835 73.92651329638639 L312.12870117707337 81.91806163143306 L317.0870602108633 85.94199262364359 L322.04541924465315 81.29262363711737 L327.00377827844306 75.58496741811899 L331.9621373122329 75.92455509773464 L336.92049634602284 84.43710590680732 L341.8788553798127 94.28174486866396 L346.83721441360257 102.65482472130125 L351.7955734473925 111.16676752069588 L356.75393248118235 119.80205970637988 L361.71229151497226 128.13417291127072 L366.6706505487621 136.20229092392256 L371.62900958255204 144.0535073858305 L376.5873686163419 151.7349159384893 L381.54572765013177 159.28799858786135 L386.5040866839217 166.70129781845188 L391.46244571771155 174.03055751314514 L396.42080475150146 181.10833364448644 L401.3791637852913 187.51893529421767 L406.3375228190812 193.14760356417128 L411.2958818528711 199.10110056276665 L416.25424088666097 204.98217880370368 L421.2125999204509 210.30109611146568 L426.17095895424075 215.57704802727397 L431.12931798803066 220.79306841799485 L436.0876770218205 225.94295060756926 L441.0460360556104 231.04431093067512 L446.0043950894003 236.09682216136179 L450.96275412319017 241.0997606279094 L455.9211131569801 246.05560022186302 L460.87947219077 250.9640237307281 L465.83783122455986 255.82406959515458 L470.7961902583497 260.63462822621244 L475.7545492921396 265.3949647926745 L480.71290832592956 270.10461539101726 L485.6712673597194 274.7655178373337 L490.6296263935093 279.3798807018726 L495.5879854272991

In [9]:
// compare it with `statSmooth` + usual `line`
val statSmoothAndLinePlot = plot {
    statSmooth(newXs, newYs) {
        line {
            x(Stat.x)
            y(Stat.y)
        }
    }
   layout.title = "`statSmooth()` + non-statistical `line` layer"
} 
plotGrid(listOf(smoothLineLayerPlot, statSmoothAndLinePlot))

-2
 
 
 
 
 
 
 
 
 -1
 
 
 
 
 
 
 
 
 0
 
 
 
 
 
 
 
 
 1
 
 
 
 
 
 
 
 
 2
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0.5
 
 
 
 
 
 
 1.0
 
 
 
 
 
 
 1.5
 
 
 
 
 
 
 2.0
 
 
 
 
 
 
 2.5
 
 
 
 
 
 
 3.0
 
 
 
 
 
 
 
 
 
 
 
 
 <path d="M17.634786308169026 305.4545454545455 L17.634786308169026 305.4545454545455 L21.197369400728434 300.1852028755624 L24.75995249328784 294.8918876553049 L28.32253558584722 289.5559215322541 L31.885118678406627 284.17857095745757 L35.447701770966006 278.7643198556748 L39.01028486352541 273.31248290976583 L42.57286795608482 267.8225857471122 L46.13545104864423 262.29534407952053 L49.698034141203635 256.7314151945559 L53.260617233763014 251.13143707014044 L56.82320032632242 245.49676335306657 L60.3857834188818 239.82883345714708 L63.94836651144121 234.12808342344846 L67.51094960400061 228.39619172215268 L71.07353269656 222.63831168752392 L74.63611578911942 216.8601825940333 L78.19869888167881 211.06754371615202 L81.7612819742382 205.26613432835114 L85.32386506679761 199.46216225693536 L88.886448159357 193.66776194818448 L92.44903125191641 187.90681383497454 L96.01161434447579 182.19479896412034 L99.57419743703521 176.51767752052825 L103.1367805295946 170.85698488622222 L106.699363622154 165.15500923453905 L110.2619467147134 158.9803772388593 L113.82452980727281 151.2237088918001 L117.3871128998322 142.87673351944747 L120.94969599239158 135.96563641844233 L124.51227908495099 128.3940141426522 L128.0748621775104 118.89489089479247 L131.6374452700698 108.68861196513777 L135.2000283626292 98.52659716114152 L138.7626114551886 87.7713430344734 L142.325194547748 76.98203732327724 L145.88777764030738 66.54155308771996 L149.45036073286678 55.69434342124407 L153.0129438254262 43.7063856624315 L156.5755269179856 31.908879016379956 L160.138110010545 21.179491263611055 L163.70069310310438 14.545454545454561 L167.26327619566376 18.467127747084305 L170.82585928822317 27.726103249279788 L174.38844238078258 36.00474544929119 L177.95102547334199 40.58852424724813 L181.5136085659014 42.977197003981416 L185.0761916584608 44.89707438662714 L188.63877475102018 47.73792353309966 L192.20135784357956 52.52556441435854 L195.76394093613897 56.88476636590269 L199.3265240286984 53.08951674102258 L202.88910712125778 52.97960437307745 L206.45169021381716 56.25741100243596 L210.0142733063766 57.52708179580691 L213.57685639893597 62.97676313277668 L217.13943949149538 67.40209041779892 L220.70202258405476 73.92651329638639 L224.26460567661414 81.91806163143306 L227.82718876917357 85.94199262364359 L231.38977186173295 81.29262363711737 L234.9523549542924 75.58496741811899 L238.51493804685177 75.92455509773464 L242.07752113941117 84.43710590680732 L245.64010423197055 94.28174486866396 L249.20268732452996 102.65482472130125 L252.76527041708937 111.16676752069588 L256.3278535096488 119.80205970637988 L259.8904366022082 128.13417291127072 L263.45301969476753 136.20229092392256 L267.015602787327 144.0535073858305 L270.57818587988635 151.7349159384893 L274.14076897244576 159.28799858786135 L277.70335206500516 166.70129781845188 L281.2659351575645 174.03055751314514 L284.828518250124 181.10833364448644 L288.3911013426833 187.51893529421767 L291.95368443524274 193.14760356417128 L295.51626752780214 199.10110056276665 L299.07885062036155 204.98217880370368 L302.64143371292096 210.30109611146568 L306.20401680548036 215.57704802727397 L309.7665998980398 220.79306841799485 L313.3291829905991 225.94295060756926 L316.89176608315853 231.04431093067512 L320.45434917571794 236.09682216136179 L324.0169322682773 241.0997606279094 L327.57951536083675 246.05560022186302 L331.1420984533961 250.9640237307281 L334.70468154595557 255.82406959515458 L338.2672646385149 260.63462822621244 L341.8298477310743 265.3949647926745 L345.39243082363373 270.10461539101726 L348.95501391619314 274.7655178373337 L352.51759700875255 279.3798807018726 L356.08018010131195 283.94714791694713 L359.6427631938713 288.4659363953719 L363.20534628643077 292.93525270011094

`smoothLine` uses `statSmooth` and `line` and performs coordinates mappings under the hood. And of course we can customize `smoothLine` layer: `smoothLine()` optionally opens a new context, where we can configure line (as in usual context opened by `line { ... }`) - even change coordinate mappings from default ones. `StatSmooth` dataset of `smoothLine` is also can be accessed here.

In [10]:
df.plot {
    smoothLine(speed, efficiency, SmoothMethod.LOESS(span = 0.1), 120) {
         // change a column mapped on `y` to `Stat.scaled`
        y(Stat.yMax)
        color = Color.RED
        width = 4.0
    }
    points {
        x(speed)
        y(efficiency)
    }
}

-2.0
 
 
 
 
 
 
 
 
 -1.5
 
 
 
 
 
 
 
 
 -1.0
 
 
 
 
 
 
 
 
 -0.5
 
 
 
 
 
 
 
 
 0.0
 
 
 
 
 
 
 
 
 0.5
 
 
 
 
 
 
 
 
 1.0
 
 
 
 
 
 
 
 
 1.5
 
 
 
 
 
 
 
 
 2.0
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0.5
 
 
 
 
 
 
 1.0
 
 
 
 
 
 
 1.5
 
 
 
 
 
 
 2.0
 
 
 
 
 
 
 2.5
 
 
 
 
 
 
 3.0
 
 
 
 
 
 
 3.5
 
 
 
 
 
 
 4.0
 
 
 
 
 
 
 
 
 
 
 
 
 <path d="M24.54387721725996 307.13085235090836 L24.54387721725996 307.13085235090836 L28.66889859831207 307.7068358134887 L32.79391997936415 308.5694774920201 L36.918941360416255 309.94132201560535 L41.043962741468306 311.1552087207419 L45.16898412252041 310.90195397374373 L49.29400550357249 308.8889127275919 L53.4190268846246 306.83092415248143 L57.54404826567668 305.21165135973405 L61.669069646728786 302.85510705525155 L65.79409102778087 298.64054147812675 L69.91911240883294 294.3929838744584 L74.04413378988502 291.8636818256928 L78.16915517093713 290.82415506476013 L82.29417655198924 289.9998154670197 L86.41919793304132 289.1274124413365 L90.5442193140934 288.01273567883675 L94.66924069514548 286.58403115304577 L98.79426207619758 284.882780606258 L102.91928345724966 282.9547227264158 L107.04430483830174 280.84559538767974 L111.16932621935385 278.60113561229946 L115.29434760040596 276.2670795319905 L119.41936898145804 273.7082162181994 L123.54439036251011 270.5168791994015 L127.66941174356222 269.7273386790308 L131.7944331246143 273.7378505115028 L135.91945450566638 276.06395488027204 L140.0444758867185 268.06413091197527 L144.16949726777057 243.532440257504 L148.29451864882265 240.2647107050027 L152.41954002987472 232.93134585771173 L156.54456141092683 224.62732664111476 L160.66958279197894 220.57122872006374 L164.79460417303102 215.3220243538802 L168.9196255540831 203.5137540283297 L173.04464693513518 192.02023809738768 L177.1696683161873 188.32484692097785 L181.2946896972394 185.99785565718204 L185.41971107829147 185.74336178671834 L189.54473245934355 176.76073323445112 L193.66975384039566 161.78989734665046 L197.7947752214477 146.9957309334655 L201.91979660249984 137.59602854641193 L206.0448179835519 130.701540799376 L210.169839364604 119.59945177396298 L214.29486074565608 110.7308585285788 L218.4198821267082 103.00670635486372 L222.5449035077603 94.63850905740793 L226.66992488881237 88.7588558355094 L230.79494626986445 79.5470604406965 L234.91996765091653 69.620921541109 L239.04498903196864 59.97515465033672 L243.17001041302072 51.54511619120734 L247.29503179407283 44.50477062267731 L251.4200531751249 38.5007308231136 L255.545074556177 33.169059814145555 L259.67009593722906 28.188183950117548 L263.7951173182812 23.374019007357276 L267.9201386993333 18.572446715475905 L272.04516008038536 16.8635763825485 L276.17018146143744 23.313812185042423 L280.2952028424895 27.028687097444276 L284.42022422354165 29.852338539488926 L288.54524560459373 31.315001274536257 L292.6702669856458 39.55643499970063 L296.79528836669795 47.62588020826939 L300.92030974774997 56.454722862311314 L305.04533112880205 65.82795633478514 L309.1703525098541 75.48338226672865 L313.29537389090626 85.91335805029712 L317.42039527195834 97.72256363326761 L321.5454166530104 111.19931294462626 L325.6704380340625 122.13968673721078 L329.79545941511464 131.50549069816097 L333.9204807961667 143.79093438183745 L338.0455021772188 155.21464944924614 L342.1705235582709 165.81825293990883 L346.295544939323 176.0235941521986 L350.4205663203751 186.30655970931937 L354.54558770142717 195.88451595069037 L358.67060908247925 203.50183993367077 L362.7956304635313 209.54818976427435 L366.9206518445834 214.59782712042755 L371.0456732256355 219.22501276409068 L375.1706946066876 224.0040064493586 L379.2957159877397 229.49171135281722 L383.4207373687918 235.0962786315306 L387.54575874984386 238.4403728674029 L391.670780130896 238.69186482497324 L395.79580151194807 238.672623364111 L399.92082289300015 242.01662795047366 L404.04584427405223 251.96684286239866 L408.1708656551043 257.9177307104758 L412.295887036156

## Grouped `statSmoth`

`statSmooth` can be applied for grouped data - statistics will be calculated on each group independently but with equal categories. This application returns a new `GroupBy` dataset with the same keys as the old one but with `StatSmooth` groups instead of old ones.

In [11]:
// generate two lines
// lines formulas
val fA = { x: Double -> 0.02 * x * x * x - 0.2 * x * x + 0.1 * x + 2.1 }
val fB = { x: Double -> -0.1 * x * x * x + 0.5 * x * x - 0.8 }
val xRange = (-500..500).map { it / 100.0 }
val noisesA = NormalDistribution(0.0, 0.05).sample(xRange.size).toList()
val noisesB = NormalDistribution(0.0, 0.2).sample(xRange.size).toList()
val valuesA = xRange.zip(noisesA).map { fA(it.first) + it.second }
val valuesB = xRange.zip(noisesB).map { fB(it.first) + it.second }

val (xsA, ysA) = xRange.zip(valuesA).shuffled(Random(17)).take(xRange.size * 1 / 3).sortedBy { it.first }.unzip()
val (xsB, ysB) = xRange.zip(valuesB).shuffled(Random(17)).take(xRange.size * 1 / 3).sortedBy { it.first }.unzip()

In [12]:
// gather them into `DataFrame` with "A" and "B" keys in "category" column
val valuesDF = dataFrameOf(
    "time" to xsA + xsB,
    "value" to ysA + ysB,
    "category" to List(xsA.size) { "A" } + List(xsB.size) { "B" }
)
valuesDF.head(5)

<iframe onload="o_resize_iframe_out_3()" style="width:100%;" class="result_container" id="iframe_out_3" frameBorder="0" srcdoc=" <html>
 <head>
 <style type="text/css">
 :root {
 --background: #fff;
 --background-odd: #f5f5f5;
 --background-hover: #d9edfd;
 --header-text-color: #474747;
 --text-color: #848484;
 --text-color-dark: #000;
 --text-color-medium: #737373;
 --text-color-pale: #b3b3b3;
 --inner-border-color: #aaa;
 --bold-border-color: #000;
 --link-color: #296eaa;
 --link-color-pale: #296eaa;
 --link-hover: #1a466c;
}

:root[theme="dark"], :root [data-jp-theme-light="false"], .dataframe_dark{
 --background: #303030;
 --background-odd: #3c3c3c;
 --background-hover: #464646;
 --header-text-color: #dddddd;
 --text-color: #b3b3b3;
 --text-color-dark: #dddddd;
 --text-color-medium: #b2b2b2;
 --text-color-pale: #737373;
 --inner-border-color: #707070;
 --bold-border-color: #777777;
 --link-color: #008dc0;
 --link-color-pale: #97e1fb;
 --link-hover: #00688e;
}

p.dataframe_description {
 color: var(--text-color-dark);
}

table.dataframe {
 font-family: "Helvetica Neue", Helvetica, Arial, sans-serif;
 font-size: 12px;
 background-color: var(--background);
 color: var(--text-color-dark);
 border: none;
 border-collapse: collapse;
}

table.dataframe th, td {
 padding: 6px;
 border: 1px solid transparent;
 text-align: left;
}

table.dataframe th {
 background-color: var(--background);
 color: var(--header-text-color);
}

table.dataframe td {
 vertical-align: top;
}

table.dataframe th.bottomBorder {
 border-bottom-color: var(--bold-border-color);
}

table.dataframe tbody > tr:nth-child(odd) {
 background: var(--background-odd);
}

table.dataframe tbody > tr:nth-child(even) {
 background: var(--background);
}

table.dataframe tbody > tr:hover {
 background: var(--background-hover);
}

table.dataframe a {
 cursor: pointer;
 color: var(--link-color);
 text-decoration: none;
}

table.dataframe tr:hover > td a {
 color: var(--link-color-pale);
}

table.dataframe a:hover {
 color: var(--link-hover);
 text-decoration: underline;
}

table.dataframe img {
 max-width: fit-content;
}

table.dataframe th.complex {
 background-color: var(--background);
 border: 1px solid var(--background);
}

table.dataframe .leftBorder {
 border-left-color: var(--inner-border-color);
}

table.dataframe .rightBorder {
 border-right-color: var(--inner-border-color);
}

table.dataframe .rightAlign {
 text-align: right;
}

table.dataframe .expanderSvg {
 width: 8px;
 height: 8px;
 margin-right: 3px;
}

table.dataframe .expander {
 display: flex;
 align-items: center;
}

/* formatting */

table.dataframe .null {
 color: var(--text-color-pale);
}

table.dataframe .structural {
 color: var(--text-color-medium);
 font-weight: bold;
}

table.dataframe .dataFrameCaption {
 font-weight: bold;
}

table.dataframe .numbers {
 color: var(--text-color-dark);
}

table.dataframe td:hover .formatted .structural, .null {
 color: var(--text-color-dark);
}

table.dataframe tr:hover .formatted .structural, .null {
 color: var(--text-color-dark);
}




:root {
 --scroll-bg: #f5f5f5;
 --scroll-fg: #b3b3b3;
}
:root[theme="dark"], :root [data-jp-theme-light="false"]{
 --scroll-bg: #3c3c3c;
 --scroll-fg: #97e1fb;
}
body {
 scrollbar-color: var(--scroll-fg) var(--scroll-bg);
}
body::-webkit-scrollbar {
 width: 10px; /* Mostly for vertical scrollbars */
 height: 10px; /* Mostly for horizontal scrollbars */
}
body::-webkit-scrollbar-thumb {
 background-color: var(--scroll-fg);
}
body::-webkit-scrollbar-track {
 background-color: var(--scroll-bg);
}
 </style>
 </head>
 <body>
 
<table class="dataframe" id="df_939524098"></table>

<p class="dataframe_description">DataFrame: rowsCount = 5, columnsCount = 3</p>

 </body>
 <script>
 (function () {
 window.DataFrame = window.DataFrame || new (function () {
 this.addTable = function (df) {
 let cols = df.cols;
 for (let i = 0; i < cols.length; i++) {
 for (let c of cols[i].children) {
 cols[c].parent = i;
 }
 }
 df.nrow = 0
 for (let i = 0; i 

It has the following signature:

<table>
  <thead>
    <tr>
      <th>time</th>
      <th>value</th>
      <th>category</th>
    </tr>
  </thead>
</table>

In [13]:
// group it by "category"
val groupedDF = valuesDF.groupBy { category }
groupedDF

<iframe onload="o_resize_iframe_out_4()" style="width:100%;" class="result_container" id="iframe_out_4" frameBorder="0" srcdoc=" <html>
 <head>
 <style type="text/css">
 :root {
 --background: #fff;
 --background-odd: #f5f5f5;
 --background-hover: #d9edfd;
 --header-text-color: #474747;
 --text-color: #848484;
 --text-color-dark: #000;
 --text-color-medium: #737373;
 --text-color-pale: #b3b3b3;
 --inner-border-color: #aaa;
 --bold-border-color: #000;
 --link-color: #296eaa;
 --link-color-pale: #296eaa;
 --link-hover: #1a466c;
}

:root[theme="dark"], :root [data-jp-theme-light="false"], .dataframe_dark{
 --background: #303030;
 --background-odd: #3c3c3c;
 --background-hover: #464646;
 --header-text-color: #dddddd;
 --text-color: #b3b3b3;
 --text-color-dark: #dddddd;
 --text-color-medium: #b2b2b2;
 --text-color-pale: #737373;
 --inner-border-color: #707070;
 --bold-border-color: #777777;
 --link-color: #008dc0;
 --link-color-pale: #97e1fb;
 --link-hover: #00688e;
}

p.dataframe_description {
 color: var(--text-color-dark);
}

table.dataframe {
 font-family: "Helvetica Neue", Helvetica, Arial, sans-serif;
 font-size: 12px;
 background-color: var(--background);
 color: var(--text-color-dark);
 border: none;
 border-collapse: collapse;
}

table.dataframe th, td {
 padding: 6px;
 border: 1px solid transparent;
 text-align: left;
}

table.dataframe th {
 background-color: var(--background);
 color: var(--header-text-color);
}

table.dataframe td {
 vertical-align: top;
}

table.dataframe th.bottomBorder {
 border-bottom-color: var(--bold-border-color);
}

table.dataframe tbody > tr:nth-child(odd) {
 background: var(--background-odd);
}

table.dataframe tbody > tr:nth-child(even) {
 background: var(--background);
}

table.dataframe tbody > tr:hover {
 background: var(--background-hover);
}

table.dataframe a {
 cursor: pointer;
 color: var(--link-color);
 text-decoration: none;
}

table.dataframe tr:hover > td a {
 color: var(--link-color-pale);
}

table.dataframe a:hover {
 color: var(--link-hover);
 text-decoration: underline;
}

table.dataframe img {
 max-width: fit-content;
}

table.dataframe th.complex {
 background-color: var(--background);
 border: 1px solid var(--background);
}

table.dataframe .leftBorder {
 border-left-color: var(--inner-border-color);
}

table.dataframe .rightBorder {
 border-right-color: var(--inner-border-color);
}

table.dataframe .rightAlign {
 text-align: right;
}

table.dataframe .expanderSvg {
 width: 8px;
 height: 8px;
 margin-right: 3px;
}

table.dataframe .expander {
 display: flex;
 align-items: center;
}

/* formatting */

table.dataframe .null {
 color: var(--text-color-pale);
}

table.dataframe .structural {
 color: var(--text-color-medium);
 font-weight: bold;
}

table.dataframe .dataFrameCaption {
 font-weight: bold;
}

table.dataframe .numbers {
 color: var(--text-color-dark);
}

table.dataframe td:hover .formatted .structural, .null {
 color: var(--text-color-dark);
}

table.dataframe tr:hover .formatted .structural, .null {
 color: var(--text-color-dark);
}




:root {
 --scroll-bg: #f5f5f5;
 --scroll-fg: #b3b3b3;
}
:root[theme="dark"], :root [data-jp-theme-light="false"]{
 --scroll-bg: #3c3c3c;
 --scroll-fg: #97e1fb;
}
body {
 scrollbar-color: var(--scroll-fg) var(--scroll-bg);
}
body::-webkit-scrollbar {
 width: 10px; /* Mostly for vertical scrollbars */
 height: 10px; /* Mostly for horizontal scrollbars */
}
body::-webkit-scrollbar-thumb {
 background-color: var(--scroll-fg);
}
body::-webkit-scrollbar-track {
 background-color: var(--scroll-bg);
}
 </style>
 </head>
 <body>
 
<table class="dataframe" id="df_939524099"></table>

<p class="dataframe_description">GroupBy</p>

 </body>
 <script>
 (function () {
 window.DataFrame = window.DataFrame || new (function () {
 this.addTable = function (df) {
 let cols = df.cols;
 for (let i = 0; i < cols.length; i++) {
 for (let c of cols[i].children) {
 cols[c].parent = i;
 }
 }
 df.nrow = 0
 for (let i = 0; i < df.cols.length; i++) {
 if (df.co

Now we have a `GroupBy` with a signature

<table>
  <thead>
    <tr>
      <th>key: [category]</th>
      <th>group: DataFrame[time|value|category]</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <td>A</td>
      <td>A-Group</td>
    </tr>
    <tr>
      <td>B</td>
      <td>B-Group</td>
    </tr>
  </tbody>
</table>

In [14]:
groupedDF.statSmooth { x(time); y(value) }

<iframe onload="o_resize_iframe_out_5()" style="width:100%;" class="result_container" id="iframe_out_5" frameBorder="0" srcdoc=" <html>
 <head>
 <style type="text/css">
 :root {
 --background: #fff;
 --background-odd: #f5f5f5;
 --background-hover: #d9edfd;
 --header-text-color: #474747;
 --text-color: #848484;
 --text-color-dark: #000;
 --text-color-medium: #737373;
 --text-color-pale: #b3b3b3;
 --inner-border-color: #aaa;
 --bold-border-color: #000;
 --link-color: #296eaa;
 --link-color-pale: #296eaa;
 --link-hover: #1a466c;
}

:root[theme="dark"], :root [data-jp-theme-light="false"], .dataframe_dark{
 --background: #303030;
 --background-odd: #3c3c3c;
 --background-hover: #464646;
 --header-text-color: #dddddd;
 --text-color: #b3b3b3;
 --text-color-dark: #dddddd;
 --text-color-medium: #b2b2b2;
 --text-color-pale: #737373;
 --inner-border-color: #707070;
 --bold-border-color: #777777;
 --link-color: #008dc0;
 --link-color-pale: #97e1fb;
 --link-hover: #00688e;
}

p.dataframe_description {
 color: var(--text-color-dark);
}

table.dataframe {
 font-family: "Helvetica Neue", Helvetica, Arial, sans-serif;
 font-size: 12px;
 background-color: var(--background);
 color: var(--text-color-dark);
 border: none;
 border-collapse: collapse;
}

table.dataframe th, td {
 padding: 6px;
 border: 1px solid transparent;
 text-align: left;
}

table.dataframe th {
 background-color: var(--background);
 color: var(--header-text-color);
}

table.dataframe td {
 vertical-align: top;
}

table.dataframe th.bottomBorder {
 border-bottom-color: var(--bold-border-color);
}

table.dataframe tbody > tr:nth-child(odd) {
 background: var(--background-odd);
}

table.dataframe tbody > tr:nth-child(even) {
 background: var(--background);
}

table.dataframe tbody > tr:hover {
 background: var(--background-hover);
}

table.dataframe a {
 cursor: pointer;
 color: var(--link-color);
 text-decoration: none;
}

table.dataframe tr:hover > td a {
 color: var(--link-color-pale);
}

table.dataframe a:hover {
 color: var(--link-hover);
 text-decoration: underline;
}

table.dataframe img {
 max-width: fit-content;
}

table.dataframe th.complex {
 background-color: var(--background);
 border: 1px solid var(--background);
}

table.dataframe .leftBorder {
 border-left-color: var(--inner-border-color);
}

table.dataframe .rightBorder {
 border-right-color: var(--inner-border-color);
}

table.dataframe .rightAlign {
 text-align: right;
}

table.dataframe .expanderSvg {
 width: 8px;
 height: 8px;
 margin-right: 3px;
}

table.dataframe .expander {
 display: flex;
 align-items: center;
}

/* formatting */

table.dataframe .null {
 color: var(--text-color-pale);
}

table.dataframe .structural {
 color: var(--text-color-medium);
 font-weight: bold;
}

table.dataframe .dataFrameCaption {
 font-weight: bold;
}

table.dataframe .numbers {
 color: var(--text-color-dark);
}

table.dataframe td:hover .formatted .structural, .null {
 color: var(--text-color-dark);
}

table.dataframe tr:hover .formatted .structural, .null {
 color: var(--text-color-dark);
}




:root {
 --scroll-bg: #f5f5f5;
 --scroll-fg: #b3b3b3;
}
:root[theme="dark"], :root [data-jp-theme-light="false"]{
 --scroll-bg: #3c3c3c;
 --scroll-fg: #97e1fb;
}
body {
 scrollbar-color: var(--scroll-fg) var(--scroll-bg);
}
body::-webkit-scrollbar {
 width: 10px; /* Mostly for vertical scrollbars */
 height: 10px; /* Mostly for horizontal scrollbars */
}
body::-webkit-scrollbar-thumb {
 background-color: var(--scroll-fg);
}
body::-webkit-scrollbar-track {
 background-color: var(--scroll-bg);
}
 </style>
 </head>
 <body>
 
<table class="dataframe" id="df_939524102"></table>

<p class="dataframe_description">GroupBy</p>

 </body>
 <script>
 (function () {
 window.DataFrame = window.DataFrame || new (function () {
 this.addTable = function (df) {
 let cols = df.cols;
 for (let i = 0; i < cols.length; i++) {
 for (let c of cols[i].children) {
 cols[c].parent = i;
 }
 }
 df.nrow = 0
 for (let i = 0; i < df.cols.length; i++) {
 if (df.co

After `statSmooth` applying it's still a `GroupBy` but with different signature of `group` - all groups have the same signature as usual `DataFrame` after `statSmooth` applying (i.e. `StatSmoothFrame`):

<table>
  <thead>
    <tr>
      <th>key: [drv]</th>
      <th>group: StaSmoothFrame</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <td>"A"</td>
      <td>"A"-Group</td>
    </tr>
    <tr>
      <td>"B"</td>
      <td>"B"-Group</td>
    </tr>
  </tbody>
</table>

As you can see, we did indeed do a `statSmooth` transformation within groups, the grouping keys did not change.

The plotting process doesn't change much - we do everything the same. 

In [15]:
groupedDF.plot {
    statSmooth(time, value) {
        line {
            x(Stat.x)
            y(Stat.y)
        }
    }
}

-4
 
 
 
 
 
 
 
 
 -2
 
 
 
 
 
 
 
 
 0
 
 
 
 
 
 
 
 
 2
 
 
 
 
 
 
 
 
 4
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 -5
 
 
 
 
 
 
 0
 
 
 
 
 
 
 5
 
 
 
 
 
 
 10
 
 
 
 
 
 
 15
 
 
 
 
 
 
 
 
 
 
 
 
 <path d="M24.77449041313926 328.3636363636364 L24.77449041313926 328.3636363636364 L29.77943797134921 325.77942261050543 L34.78438552955916 323.20368884191424 L39.78933308776911 320.6370593148187 L44.79428064597906 318.0802015850197 L49.799228204189006 315.5338493167276 L54.804175762398955 312.9987850679592 L59.809123320608904 310.4758206720624 L64.81407087881885 307.9657419508438 L69.8190184370288 305.4691889125942 L74.82396599523875 302.9865329582093 L79.8289135534487 300.51787173124046 L84.83386111165868 298.0630938661895 L89.83880866986863 295.6218915816902 L94.84375622807858 293.19379959276034 L99.84870378628852 290.7782422144584 L104.85365134449847 288.37457483031943 L109.85859890270842 285.98209962102806 L114.86354646091837 283.5999719177856 L119.86849401912832 281.2272012042766 L124.87344157733827 278.8626024612269 L129.87838913554822 276.5047429797245 L134.88333669375817 274.1517893858386 L139.88828425196812 271.8013975092051 L144.89323181017807 269.445415040514 L149.898179368388 267.1975011969041 L154.90312692659796 265.0886673513577 L159.9080744848079 262.9580794187962 L164.91302204301786 261.12586744797034 L169.9179696012278 259.4233987639784 L174.92291715943776 257.45928316013914 L179.9278647176477 255.66749136831487 L184.93281227585766 254.0277579490565 L189.9377598340676 252.35457889756347 L194.94270739227755 251.0019676911066 L199.9476549504875 249.91289433035317 L204.95260250869745 248.7414224104426 L209.9575500669074 247.70400027728385 L214.96249762511735 246.70793323492347 L219.9674451833273 245.67704222331062 L224.97239274153725 244.86077306053085 L229.9773402997472 243.98308661967008 L234.98228785795715 242.9030896258188 L239.9872354161671 242.30460644461456 L244.99218297437704 241.7618442806911 L249.997130532587 241.21168554783006 L255.00207809079694 240.932075376868 L260.0070256490069 240.58674426266865 L265.0119732072169 240.22660543217168 L270.01692076542685 240.1958495089429 L275.0218683236368 239.94764725537556 L280.02681588184674 239.8797367620679 L285.0317634400567 239.95745626481522 L290.03671099826664 239.98702619984616 L295.0416585564766 240.04404483261524 L300.04660611468654 240.07547726905884 L305.0515536728965 240.09901488880826 L310.05650123110644 240.00468342608877 L315.0614487893164 240.33581089832703 L320.06639634752634 240.65094123706203 L325.0713439057363 240.95694696063273 L330.07629146394623 241.356310004733 L335.0812390221562 241.61799858461018 L340.08618658036613 242.01363498335726 L345.0911341385761 242.34990833299256 L350.09608169678603 242.77701600396998 L355.101029254996 243.18574279841746 L360.1059768132059 243.7171253196467 L365.1109243714159 244.27039109221403 L370.1158719296258 244.81151505017237 L375.1208194878358 245.34293997617218 L380.1257670460457 245.90510957528716 L385.1307146042557 246.5731681393774 L390.1356621624656 247.1944614908268 L395.14060972067557 247.88894337534018 L400.1455572788855 248.60661934095168 L405.15050483709547 249.30641931741224 L410.1554523953054 249.99522409676672 L415.16039995351537 250.74905203909708 L420.1653475117253 251.50263505603868 L425.17029506993526 252.25640569794055 L430.17524262814527 253.0107463065098 L435.18019018635516 253.7658874450365 L440.18513774456517 254.52195643132035 L445.19008530277506 255.2790099348839 L450.19503286098507 256.03707026496085 L455.19998041919496 256.79613797634795 L460.20492797740496 257.5561933477096 L465.20987553561486 258.31719255422195 L470.21482309382486 259.07907320465273 L475.21977065203475 259.84176450404755 L480.22471821024476 260.60518818732425 L485.22966576845465 261.3692621680037 L490.23461332666466 262.13390245565495 L495.23956088487455 262.8990345460604 L500.24450844308456 263.66458510381517 L505.24945600129456 264.43048488860694 L510.25440355950445 265.1966635977741 L515.2593511177145 2

As you can see, there are two lines because we have two groups of data. To distinguish them, we need to add mapping to the color from the key. This is easy - key is available in the context

In [16]:
groupedDF.plot {
    statSmooth(time, value, method = SmoothMethod.Polynomial(3)) {
        line {
            x(Stat.x)
            y(Stat.y)
            color(key.category)
        }
    }
}

-4
 
 
 
 
 
 
 
 
 -2
 
 
 
 
 
 
 
 
 0
 
 
 
 
 
 
 
 
 2
 
 
 
 
 
 
 
 
 4
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 -5
 
 
 
 
 
 
 0
 
 
 
 
 
 
 5
 
 
 
 
 
 
 10
 
 
 
 
 
 
 15
 
 
 
 
 
 
 20
 
 
 
 
 
 
 25
 
 
 
 
 
 
 
 
 
 
 
 
 <path d="M20.82183586673196 328.3636363636364 L20.82183586673196 328.3636363636364 L25.028267354960633 324.6029445327832 L29.234698843189307 320.9479675261083 L33.44113033141798 317.3974114350156 L37.64756181964668 313.9499823509094 L41.853993307875356 310.60438636519376 L46.06042479610403 307.359329569273 L50.266856284332704 304.21351805455106 L54.47328777256138 301.16565791243227 L58.67971926079005 298.2144552343207 L62.886150749018725 295.35861611162056 L67.0925822372474 292.59684663573603 L71.2990137254761 289.9278528980712 L75.50544521370477 287.35034099003025 L79.71187670193345 284.8630170030174 L83.91830819016212 282.4645870284367 L88.1247396783908 280.1537571576924 L92.33117116661947 277.9292334821886 L96.53760265484814 275.7897220933295 L100.74403414307683 273.73392908251924 L104.9504656313055 271.760560541162 L109.15689711953418 269.8683225606619 L113.36332860776285 268.05592123242315 L117.56976009599154 266.32206264784986 L121.77619158422023 264.66545289834625 L125.9826230724489 263.08479807531637 L130.18905456067756 261.57880427016454 L134.39548604890626 260.14617757429477 L138.60191753713494 258.7856240791113 L142.8083490253636 257.49584987601827 L147.01478051359229 256.2755610564198 L151.22121200182096 255.12346371172012 L155.42764349004963 254.03826393332335 L159.6340749782783 253.01866781263365 L163.840506466507 252.06338144105518 L168.04693795473565 251.1711109099921 L172.25336944296436 250.34056231084858 L176.459800931193 249.5704417350288 L180.6662324194217 248.85945527393685 L184.87266390765038 248.20630901897692 L189.07909539587905 247.60970906155322 L193.28552688410772 247.06836149306983 L197.4919583723364 246.58097240493098 L201.69838986056507 246.14624788854078 L205.90482134879375 245.76289403530342 L210.11125283702242 245.42961693662303 L214.3176843252511 245.14512268390376 L218.52411581347977 244.90811736854985 L222.7305473017085 244.71730708196537 L226.93697878993717 244.5713979155545 L231.14341027816585 244.46909596072143 L235.34984176639452 244.4091073088703 L239.5562732546232 244.39013805140527 L243.76270474285187 244.41089427973048 L247.96913623108054 244.47008208525014 L252.1755677193092 244.56640755936834 L256.3819992075379 244.6985767934893 L260.5884306957666 244.86529587901717 L264.79486218399524 245.06527090735608 L269.00129367222394 245.29720796991018 L273.2077251604526 245.55981315808367 L277.4141566486813 245.8517925632807 L281.62058813691 246.1718522769054 L285.82701962513863 246.51869839036198 L290.0334511133673 246.89103699505455 L294.239882601596 247.28757418238732 L298.4463140898247 247.70701604376438 L302.6527455780533 248.14806867058994 L306.85917706628203 248.60943815426816 L311.0656085545107 249.08983058620316 L315.2720400427394 249.58795205779916 L319.478471530968 250.10250866046024 L323.6849030191967 250.63220648559064 L327.8913345074254 251.17575162459448 L332.0977659956541 251.7318501688759 L336.3041974838827 252.2992082098391 L340.5106289721114 252.8765318388882 L344.7170604603401 253.4625271474274 L348.92349194856877 254.05590022686084 L353.12992343679747 254.65535716859267 L357.3363549250261 255.25960406402706 L361.5427864132548 255.86734700456816 L365.74921790148346 256.47729208162013 L369.9556493897122 257.08814538658714 L374.16208087794087 257.6986130108734 L378.3685123661695 258.30740104588295 L382.57494385439816 258.91321558302 L386.7813753426269 259.51476271368875 L390.98780683085556 260.11074852929335 L395.1942383190842 260.6998791212379 L399.40066980731285 261.2808605809266 L403.6071012955416 261.85239899976364 L407.81353278377026 262.41320046915314 L412.01996427199896 262.96197108049927 L416.2263957602276 263.49741692520615 L420.4328272484563 264.01824409467804 L424.639258736685 264.523158680319 L428.84569022491365 265.01

The `smoothLine()` layer also works. Moreover, if we have exactly one grouping key, a mapping from it to `color` will be created by default.

In [17]:
groupedDF.plot {
    smoothLine(time, value)
}

-4
 
 
 
 
 
 
 
 
 -2
 
 
 
 
 
 
 
 
 0
 
 
 
 
 
 
 
 
 2
 
 
 
 
 
 
 
 
 4
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 -5
 
 
 
 
 
 
 0
 
 
 
 
 
 
 5
 
 
 
 
 
 
 10
 
 
 
 
 
 
 15
 
 
 
 
 
 
 
 
 
 
 
 
 <path d="M20.82183586673196 328.3636363636364 L20.82183586673196 328.3636363636364 L25.028267354960633 325.77942261050543 L29.234698843189307 323.20368884191424 L33.44113033141798 320.6370593148187 L37.64756181964668 318.0802015850197 L41.853993307875356 315.5338493167276 L46.06042479610403 312.9987850679592 L50.266856284332704 310.4758206720624 L54.47328777256138 307.9657419508438 L58.67971926079005 305.4691889125942 L62.886150749018725 302.9865329582093 L67.0925822372474 300.51787173124046 L71.2990137254761 298.0630938661895 L75.50544521370477 295.6218915816902 L79.71187670193345 293.19379959276034 L83.91830819016212 290.7782422144584 L88.1247396783908 288.37457483031943 L92.33117116661947 285.98209962102806 L96.53760265484814 283.5999719177856 L100.74403414307683 281.2272012042766 L104.9504656313055 278.8626024612269 L109.15689711953418 276.5047429797245 L113.36332860776285 274.1517893858386 L117.56976009599154 271.8013975092051 L121.77619158422023 269.445415040514 L125.9826230724489 267.1975011969041 L130.18905456067756 265.0886673513577 L134.39548604890626 262.9580794187962 L138.60191753713494 261.12586744797034 L142.8083490253636 259.4233987639784 L147.01478051359229 257.45928316013914 L151.22121200182096 255.66749136831487 L155.42764349004963 254.0277579490565 L159.6340749782783 252.35457889756347 L163.840506466507 251.0019676911066 L168.04693795473565 249.91289433035317 L172.25336944296436 248.7414224104426 L176.459800931193 247.70400027728385 L180.6662324194217 246.70793323492347 L184.87266390765038 245.67704222331062 L189.07909539587905 244.86077306053085 L193.28552688410772 243.98308661967008 L197.4919583723364 242.9030896258188 L201.69838986056507 242.30460644461456 L205.90482134879375 241.7618442806911 L210.11125283702242 241.21168554783006 L214.3176843252511 240.932075376868 L218.52411581347977 240.58674426266865 L222.7305473017085 240.22660543217168 L226.93697878993717 240.1958495089429 L231.14341027816585 239.94764725537556 L235.34984176639452 239.8797367620679 L239.5562732546232 239.95745626481522 L243.76270474285187 239.98702619984616 L247.96913623108054 240.04404483261524 L252.1755677193092 240.07547726905884 L256.3819992075379 240.09901488880826 L260.5884306957666 240.00468342608877 L264.79486218399524 240.33581089832703 L269.00129367222394 240.65094123706203 L273.2077251604526 240.95694696063273 L277.4141566486813 241.356310004733 L281.62058813691 241.61799858461018 L285.82701962513863 242.01363498335726 L290.0334511133673 242.34990833299256 L294.239882601596 242.77701600396998 L298.4463140898247 243.18574279841746 L302.6527455780533 243.7171253196467 L306.85917706628203 244.27039109221403 L311.0656085545107 244.81151505017237 L315.2720400427394 245.34293997617218 L319.478471530968 245.90510957528716 L323.6849030191967 246.5731681393774 L327.8913345074254 247.1944614908268 L332.0977659956541 247.88894337534018 L336.3041974838827 248.60661934095168 L340.5106289721114 249.30641931741224 L344.7170604603401 249.99522409676672 L348.92349194856877 250.74905203909708 L353.12992343679747 251.50263505603868 L357.3363549250261 252.25640569794055 L361.5427864132548 253.0107463065098 L365.74921790148346 253.7658874450365 L369.9556493897122 254.52195643132035 L374.16208087794087 255.2790099348839 L378.3685123661695 256.03707026496085 L382.57494385439816 256.79613797634795 L386.7813753426269 257.5561933477096 L390.98780683085556 258.31719255422195 L395.1942383190842 259.07907320465273 L399.40066980731285 259.84176450404755 L403.6071012955416 260.60518818732425 L407.81353278377026 261.3692621680037 L412.01996427199896 262.13390245565495 L416.2263957602276 262.8990345460604 L420.4328272484563 263.66458510381517 L424.639258736685 264.43048488860694 L428.84569022491365 265.1966635977741 L433.05212171314236 265.9630501173

We can customize it like we used to. From the differences — access to `key` columns:

In [18]:
groupedDF.plot {
    smoothLine(time, value) {
        color = Color.GREEN
        type(key.category)
    }
}//.toLetsPlot()

-4
 
 
 
 
 
 
 
 
 -2
 
 
 
 
 
 
 
 
 0
 
 
 
 
 
 
 
 
 2
 
 
 
 
 
 
 
 
 4
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 -5
 
 
 
 
 
 
 0
 
 
 
 
 
 
 5
 
 
 
 
 
 
 10
 
 
 
 
 
 
 15
 
 
 
 
 
 
 
 
 
 
 
 
 <path d="M20.82183586673196 328.3636363636364 L20.82183586673196 328.3636363636364 L25.028267354960633 325.77942261050543 L29.234698843189307 323.20368884191424 L33.44113033141798 320.6370593148187 L37.64756181964668 318.0802015850197 L41.853993307875356 315.5338493167276 L46.06042479610403 312.9987850679592 L50.266856284332704 310.4758206720624 L54.47328777256138 307.9657419508438 L58.67971926079005 305.4691889125942 L62.886150749018725 302.9865329582093 L67.0925822372474 300.51787173124046 L71.2990137254761 298.0630938661895 L75.50544521370477 295.6218915816902 L79.71187670193345 293.19379959276034 L83.91830819016212 290.7782422144584 L88.1247396783908 288.37457483031943 L92.33117116661947 285.98209962102806 L96.53760265484814 283.5999719177856 L100.74403414307683 281.2272012042766 L104.9504656313055 278.8626024612269 L109.15689711953418 276.5047429797245 L113.36332860776285 274.1517893858386 L117.56976009599154 271.8013975092051 L121.77619158422023 269.445415040514 L125.9826230724489 267.1975011969041 L130.18905456067756 265.0886673513577 L134.39548604890626 262.9580794187962 L138.60191753713494 261.12586744797034 L142.8083490253636 259.4233987639784 L147.01478051359229 257.45928316013914 L151.22121200182096 255.66749136831487 L155.42764349004963 254.0277579490565 L159.6340749782783 252.35457889756347 L163.840506466507 251.0019676911066 L168.04693795473565 249.91289433035317 L172.25336944296436 248.7414224104426 L176.459800931193 247.70400027728385 L180.6662324194217 246.70793323492347 L184.87266390765038 245.67704222331062 L189.07909539587905 244.86077306053085 L193.28552688410772 243.98308661967008 L197.4919583723364 242.9030896258188 L201.69838986056507 242.30460644461456 L205.90482134879375 241.7618442806911 L210.11125283702242 241.21168554783006 L214.3176843252511 240.932075376868 L218.52411581347977 240.58674426266865 L222.7305473017085 240.22660543217168 L226.93697878993717 240.1958495089429 L231.14341027816585 239.94764725537556 L235.34984176639452 239.8797367620679 L239.5562732546232 239.95745626481522 L243.76270474285187 239.98702619984616 L247.96913623108054 240.04404483261524 L252.1755677193092 240.07547726905884 L256.3819992075379 240.09901488880826 L260.5884306957666 240.00468342608877 L264.79486218399524 240.33581089832703 L269.00129367222394 240.65094123706203 L273.2077251604526 240.95694696063273 L277.4141566486813 241.356310004733 L281.62058813691 241.61799858461018 L285.82701962513863 242.01363498335726 L290.0334511133673 242.34990833299256 L294.239882601596 242.77701600396998 L298.4463140898247 243.18574279841746 L302.6527455780533 243.7171253196467 L306.85917706628203 244.27039109221403 L311.0656085545107 244.81151505017237 L315.2720400427394 245.34293997617218 L319.478471530968 245.90510957528716 L323.6849030191967 246.5731681393774 L327.8913345074254 247.1944614908268 L332.0977659956541 247.88894337534018 L336.3041974838827 248.60661934095168 L340.5106289721114 249.30641931741224 L344.7170604603401 249.99522409676672 L348.92349194856877 250.74905203909708 L353.12992343679747 251.50263505603868 L357.3363549250261 252.25640569794055 L361.5427864132548 253.0107463065098 L365.74921790148346 253.7658874450365 L369.9556493897122 254.52195643132035 L374.16208087794087 255.2790099348839 L378.3685123661695 256.03707026496085 L382.57494385439816 256.79613797634795 L386.7813753426269 257.5561933477096 L390.98780683085556 258.31719255422195 L395.1942383190842 259.07907320465273 L399.40066980731285 259.84176450404755 L403.6071012955416 260.60518818732425 L407.81353278377026 261.3692621680037 L412.01996427199896 262.13390245565495 L416.2263957602276 262.8990345460604 L420.4328272484563 263.66458510381517 L424.639258736685 264.43048488860694 L428.84569022491365 265.1966635977741 L433.05212171314236 265.9630501173

### Inside `groupBy{}` plot context

We can apply `groupBy` modification to the initial dataset and build a histogram with grouped data the same way:

In [19]:
valuesDF.plot {
    groupBy(category) {
        smoothLine(time, value)
    }
}

-4
 
 
 
 
 
 
 
 
 -2
 
 
 
 
 
 
 
 
 0
 
 
 
 
 
 
 
 
 2
 
 
 
 
 
 
 
 
 4
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 -5
 
 
 
 
 
 
 0
 
 
 
 
 
 
 5
 
 
 
 
 
 
 10
 
 
 
 
 
 
 15
 
 
 
 
 
 
 
 
 
 
 
 
 <path d="M20.82183586673196 328.3636363636364 L20.82183586673196 328.3636363636364 L25.028267354960633 325.77942261050543 L29.234698843189307 323.20368884191424 L33.44113033141798 320.6370593148187 L37.64756181964668 318.0802015850197 L41.853993307875356 315.5338493167276 L46.06042479610403 312.9987850679592 L50.266856284332704 310.4758206720624 L54.47328777256138 307.9657419508438 L58.67971926079005 305.4691889125942 L62.886150749018725 302.9865329582093 L67.0925822372474 300.51787173124046 L71.2990137254761 298.0630938661895 L75.50544521370477 295.6218915816902 L79.71187670193345 293.19379959276034 L83.91830819016212 290.7782422144584 L88.1247396783908 288.37457483031943 L92.33117116661947 285.98209962102806 L96.53760265484814 283.5999719177856 L100.74403414307683 281.2272012042766 L104.9504656313055 278.8626024612269 L109.15689711953418 276.5047429797245 L113.36332860776285 274.1517893858386 L117.56976009599154 271.8013975092051 L121.77619158422023 269.445415040514 L125.9826230724489 267.1975011969041 L130.18905456067756 265.0886673513577 L134.39548604890626 262.9580794187962 L138.60191753713494 261.12586744797034 L142.8083490253636 259.4233987639784 L147.01478051359229 257.45928316013914 L151.22121200182096 255.66749136831487 L155.42764349004963 254.0277579490565 L159.6340749782783 252.35457889756347 L163.840506466507 251.0019676911066 L168.04693795473565 249.91289433035317 L172.25336944296436 248.7414224104426 L176.459800931193 247.70400027728385 L180.6662324194217 246.70793323492347 L184.87266390765038 245.67704222331062 L189.07909539587905 244.86077306053085 L193.28552688410772 243.98308661967008 L197.4919583723364 242.9030896258188 L201.69838986056507 242.30460644461456 L205.90482134879375 241.7618442806911 L210.11125283702242 241.21168554783006 L214.3176843252511 240.932075376868 L218.52411581347977 240.58674426266865 L222.7305473017085 240.22660543217168 L226.93697878993717 240.1958495089429 L231.14341027816585 239.94764725537556 L235.34984176639452 239.8797367620679 L239.5562732546232 239.95745626481522 L243.76270474285187 239.98702619984616 L247.96913623108054 240.04404483261524 L252.1755677193092 240.07547726905884 L256.3819992075379 240.09901488880826 L260.5884306957666 240.00468342608877 L264.79486218399524 240.33581089832703 L269.00129367222394 240.65094123706203 L273.2077251604526 240.95694696063273 L277.4141566486813 241.356310004733 L281.62058813691 241.61799858461018 L285.82701962513863 242.01363498335726 L290.0334511133673 242.34990833299256 L294.239882601596 242.77701600396998 L298.4463140898247 243.18574279841746 L302.6527455780533 243.7171253196467 L306.85917706628203 244.27039109221403 L311.0656085545107 244.81151505017237 L315.2720400427394 245.34293997617218 L319.478471530968 245.90510957528716 L323.6849030191967 246.5731681393774 L327.8913345074254 247.1944614908268 L332.0977659956541 247.88894337534018 L336.3041974838827 248.60661934095168 L340.5106289721114 249.30641931741224 L344.7170604603401 249.99522409676672 L348.92349194856877 250.74905203909708 L353.12992343679747 251.50263505603868 L357.3363549250261 252.25640569794055 L361.5427864132548 253.0107463065098 L365.74921790148346 253.7658874450365 L369.9556493897122 254.52195643132035 L374.16208087794087 255.2790099348839 L378.3685123661695 256.03707026496085 L382.57494385439816 256.79613797634795 L386.7813753426269 257.5561933477096 L390.98780683085556 258.31719255422195 L395.1942383190842 259.07907320465273 L399.40066980731285 259.84176450404755 L403.6071012955416 260.60518818732425 L407.81353278377026 261.3692621680037 L412.01996427199896 262.13390245565495 L416.2263957602276 262.8990345460604 L420.4328272484563 263.66458510381517 L424.639258736685 264.43048488860694 L428.84569022491365 265.1966635977741 L433.05212171314236 265.9630501173